#### 在小数据集上从头开始训练 

In [ ]:
import os, shutil

original_dataset_dir = 'C:\\Users\\nax\\Desktop\\nax\\nax-0305\\dl-things\\data\\petimages'

def create_dir_not_exists(dir_path):
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)

base_dir = 'C:\\Users\\nax\\Desktop\\nax\\nax-0305\\dl-things\\data\\dogcatsmall'
create_dir_not_exists(base_dir)
train_dir = os.path.join(base_dir, 'train')
create_dir_not_exists(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
create_dir_not_exists(validation_dir)
test_dir = os.path.join(base_dir, 'test')
create_dir_not_exists(test_dir)
train_cats_dir = os.path.join(train_dir, 'cats')
create_dir_not_exists(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
create_dir_not_exists(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
create_dir_not_exists(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
create_dir_not_exists(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
create_dir_not_exists(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
create_dir_not_exists(test_dogs_dir)

In [ ]:
original_dataset_cats_dir = original_dataset_dir + "\\cats"
original_dataset_dogs_dir = original_dataset_dir + "\\dogs"

fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_cats_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_cats_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_cats_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dogs_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dogs_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dogs_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [ ]:
 print('total training cat images:', len(os.listdir(train_cats_dir)))
 print('total training dog images:', len(os.listdir(train_dogs_dir)))
 print('total validation cat images:', len(os.listdir(validation_cats_dir)))
 print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
 print('total test cat images:', len(os.listdir(test_cats_dir)))
 print('total test dog images:', len(os.listdir(test_dogs_dir)))

In [ ]:
# 构建网络层
from keras import layers, models

# 每一层都有32个滤波器，所以会生成32个(148, 148, 1)，最后叠在一起就是(148, 148, 32)

model = models.Sequential()
model.add(layers.Input((150, 150, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [ ]:
# 检查所有图片是否可以打开
from PIL import Image

def check_image(path, begin, end):
    for i in range(begin, end):
        cat_address = os.path.join(path,  str(i) + '.jpg')
        try:
            Image.open(cat_address)
        except:
            print("Erro occur on " + cat_address)
check_image(train_cats_dir, 0, 1000)
check_image(train_dogs_dir, 0, 1000)
check_image(validation_cats_dir, 1001, 1500)
check_image(validation_dogs_dir, 1001, 1500)
check_image(test_cats_dir, 1500, 2000)
check_image(test_dogs_dir, 1500, 2000)


In [ ]:
# 数据预处理的步骤
# 读取图像文件 -> 将图像文件转为rgb像素格 -> 将其tensor化 -> 再将其数据缩放到[0, 1]

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150, 150),batch_size=20,class_mode='binary')
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
# 使用model.fit_generator训练模型
# train_generator的批次是20， train_dir下的猫狗图片一共2000张，所以需要steps_per_epoch是100次，验证集的生成器是批次20，一共1000张，所以validation是50
history = model.fit(train_generator, steps_per_epoch=100, epochs=30, validation_data=validation_generator, validation_steps=50)

In [ ]:
# 保存模型
model.save('cats_and_dogs_small_1.keras')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

### 使用数据增强来增大数据集

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
from keras.preprocessing import image
fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]
img_path = fnames[3]
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break
plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Input((150, 150, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# 不能增强验证数据
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50
)

In [ ]:
model.save('cats_and_dogs_small_2.keras')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()